In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import dask
import babet as bb
from moarpalettes import get_palette
from cartopy import crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as patches
from cmcrameri import cm
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import FuncFormatter, ScalarFormatter
from moarpalettes import get_palette

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)
# inidates = ['2023-10-11', '2023-10-15', '2023-10-17']
inidates = ['2023-10-15', '2023-10-17']
experiments = ['pi', 'curr', 'incr']

dask.config.set(**{'array.slicing.split_large_chunks': True})

In [7]:
sfc = xr.open_dataset('/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/PGW_new_runs/single_level_jexp0_expid_CTL.nc')
sfc

<xarray.Dataset>
Dimensions:  (mem: 4, height: 1, rlat: 252, rlon: 276)
Coordinates:
    lon      (rlat, rlon) float64 ...
    lat      (rlat, rlon) float64 ...
  * rlon     (rlon) float64 -15.94 -15.83 -15.72 -15.61 ... 14.09 14.2 14.31
  * rlat     (rlat) float64 -15.41 -15.3 -15.19 -15.08 ... 11.98 12.09 12.2
  * height   (height) float64 0.0
    exp      object ...
  * mem      (mem) object 'inMA00' 'inMF24' 'inMF48' 'inMF72'
Data variables:
    precip   (mem, height, rlat, rlon) float32 ...
    mslp     (mem, height, rlat, rlon) float32 ...
    qvi      (mem, height, rlat, rlon) float32 ...
Attributes:
    standard_name:  precipitation_flux
    long_name:      Total Precipitative Flux
    units:          kg m-2 s-1
    cell_methods:   time: mean (1-hr averaged values)
    grid_mapping:   rotated_pole

In [6]:
pl = xr.open_dataset('/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/PGW_new_runs/pressure_level_jexp0_expid_CTL.nc')
pl

<xarray.Dataset>
Dimensions:       (mem: 4, rlat: 252, rlon: 276, nblock1: 40, nblock2: 400, bnds: 2, pressure: 5)
Coordinates:
    lon           (rlat, rlon) float64 ...
    lat           (rlat, rlon) float64 ...
  * rlon          (rlon) float64 -15.94 -15.83 -15.72 ... 14.09 14.2 14.31
  * rlat          (rlat) float64 -15.41 -15.3 -15.19 -15.08 ... 11.98 12.09 12.2
  * pressure      (pressure) float32 3e+04 5e+04 7e+04 8.5e+04 9.25e+04
    exp           object ...
  * mem           (mem) object 'inMA00' 'inMF24' 'inMF48' 'inMF72'
Dimensions without coordinates: nblock1, nblock2, bnds
Data variables:
    dir           (mem, rlat, rlon) float64 ...
    block1        (mem, nblock1) int32 ...
    block2        (mem, nblock2) int32 ...
    dtg           (mem) float64 ...
    date_bnds     (mem, bnds) float64 ...
    hms_bnds      (mem, bnds) float64 ...
    assigned      (mem) float64 ...
    rotated_pole  (mem) float64 ...
    t             (mem, pressure, rlat, rlon) float32 ...
    wp            (mem, pressure, rlat, rlon) float32 ...
Attributes:
    Conventions:   CF-1.4
    source:        RACMO2
    Domain:        KEXT12
    Experiment:    xR2v3-FC72-inMA00-v578-CTL-fECANFC
    institution:   Royal Netherlands Meteorological Institute (KNMI)
    CreationDate:  Wed Apr 10 13:54:37 2024
    comment:       asim2cdf: cpar=t0300, iwmo=11, ilvt=100, ilev=300, idh=1, ...
    title:         Temperature 300 hPa